# Visites de la page convention collective

Dans cette exploration, le but est de savoir, comment l'ajout du lien en base des pages contributions à améliorer ou non le nombre de visites des pages `convention-collective`.

Étape 1: Avoir le % des différentes `urlref` des pages `convention-collective` avant le 11/06

Étape 2: Avoir le % des différentes `urlref` des pages `convention-collective` après le 13/06

## 1. Chargement des librairies

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

## 2. Récupération des queries sur elasticsearch

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY_BEFORE = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/convention-collective" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": "2023-05-10",
                    "lte": "2023-06-10"
               }
              }
            }
          ]
        }
      }
}


QUERY_AFTER = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/convention-collective" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": "2023-06-14",
                    "lte": "2023-07-14"
               }
              }
            }
          ]
        }
      }
}

In [ ]:
logs_before = es_connector.execute_query(QUERY_BEFORE, "logs-new")

In [ ]:
logs_after = es_connector.execute_query(QUERY_AFTER, "logs-new")

## 3. Affichage du nombre de visite unique par canaux d'acquisation

In [ ]:
logs_before.groupby(by=["referrerTypeName"]).agg({"idVisit": "nunique"})

In [ ]:
logs_after.groupby(by=["referrerTypeName"]).agg({"idVisit": "nunique"})

## 4. Traitement pour le 10 mai 2023 au 10 juin 2023

In [ ]:
# récupération seulement des events `visit_content`

logs_before_only_visit_content = logs_before[logs_before["type"] == "visit_content"] 

In [ ]:
# récuperation des urls qui auto-réfèrent cdtn

logs_before_only_from_cdtn = logs_before_only_visit_content[logs_before_only_visit_content['referrerUrl'].str.match('^https://code.travail.gouv.fr', na=False)]

In [ ]:
# Affichage du nombre de `visit_content` qui contiennent des auto lien referrer cdtn

logs_before_only_from_cdtn["url"].count()

In [ ]:
logs_before_only_from_cdtn_transformed = logs_before_only_from_cdtn.assign(slugPart=lambda x: x.referrerUrl.str.split('/').str[3].str.split('#').str[0].str.split('?').str[0])

In [ ]:
logs_before_only_from_cdtn_transformed

In [ ]:
# Si on aggrège par slug : 
logs_before_only_from_cdtn_transformed.groupby(by=["slugPart"]).size().sort_values(ascending=False)

In [ ]:
# Dans le cas, où on groupe par idVisit unique
logs_before_only_from_cdtn_transformed.groupby(by=["slugPart"]).agg({"idVisit": "nunique"}).sort_values(by=["idVisit"], ascending=False)

## 5. Traitement pour le 14 juin 2023 au 14 juillet 2023

In [ ]:
# récupération seulement des events `visit_content`

logs_after_only_visit_content = logs_after[logs_after["type"] == "visit_content"] 

In [ ]:
# récuperation des urls qui auto-réfèrent cdtn

logs_after_only_from_cdtn = logs_after_only_visit_content[logs_after_only_visit_content['referrerUrl'].str.match('^https://code.travail.gouv.fr', na=False)]

In [ ]:
# Affichage du nombre de `visit_content` qui contiennent des auto lien referrer cdtn

logs_after_only_from_cdtn["url"].count()

In [ ]:
logs_after_only_from_cdtn_transformed = logs_after_only_from_cdtn.assign(slugPart=lambda x: x.referrerUrl.str.split('/').str[3].str.split('#').str[0].str.split('?').str[0])

In [ ]:
logs_after_only_from_cdtn_transformed

In [ ]:
# Si on aggrège par slug : 
logs_after_only_from_cdtn_transformed.groupby(by=["slugPart"]).size().sort_values(ascending=False)

In [ ]:
# Dans le cas, où on groupe par idVisit unique
logs_after_only_from_cdtn_transformed.groupby(by=["slugPart"]).agg({"idVisit": "nunique"}).sort_values(by=["idVisit"], ascending=False)